<a href="https://colab.research.google.com/github/jaredmccain-ux/pytorch-tutorial-learning-notes/blob/main/_downloads1/tensorboardyt_tutorial/tensorboardyt_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For tips on running notebooks in Google Colab, see
# https://docs.pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Introduction](introyt1_tutorial.html) \|\|
[Tensors](tensors_deeper_tutorial.html) \|\|
[Autograd](autogradyt_tutorial.html) \|\| [Building
Models](modelsyt_tutorial.html) \|\| **TensorBoard Support** \|\|
[Training Models](trainingyt.html) \|\| [Model
Understanding](captumyt.html)

PyTorch TensorBoard Support
===========================

Follow along with the video below or on
[youtube](https://www.youtube.com/watch?v=6CEld3hZgqc).



In [2]:
# Run this cell to load the video
from IPython.display import display, HTML
html_code = """
<div style="margin-top:10px; margin-bottom:10px;">
  <iframe width="560" height="315" src="https://www.youtube.com/embed/6CEld3hZgqc" frameborder="0" allow="accelerometer; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</div>
"""
display(HTML(html_code))



Before You Start
----------------

To run this tutorial, you'll need to install PyTorch, TorchVision,
Matplotlib, and TensorBoard.

With `conda`:

``` {.sh}
conda install pytorch torchvision -c pytorch
conda install matplotlib tensorboard
```

With `pip`:

``` {.sh}
pip install torch torchvision matplotlib tensorboard
```

Once the dependencies are installed, restart this notebook in the Python
environment where you installed them.

Introduction
------------

In this notebook, we'll be training a variant of LeNet-5 against the
Fashion-MNIST dataset. Fashion-MNIST is a set of image tiles depicting
various garments, with ten class labels indicating the type of garment
depicted.


In [3]:
# PyTorch model and training necessities
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Image datasets and image manipulation
import torchvision
import torchvision.transforms as transforms

# Image display
import matplotlib.pyplot as plt
import numpy as np

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter

# In case you are using an environment that has TensorFlow installed,
# such as Google Colab, uncomment the following code to avoid
# a bug with saving embeddings to your TensorBoard directory

# import tensorflow as tf
# import tensorboard as tb
# tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

Showing Images in TensorBoard
=============================

Let's start by adding sample images from our dataset to TensorBoard:


100%|██████████| 26.4M/26.4M [00:01<00:00, 13.4MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.94MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 18.0MB/s]


In [4]:
# Gather datasets and prepare them for consumption
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# Store separate training and validations splits in ./data
training_set = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
validation_set = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

training_loader = torch.utils.data.DataLoader(training_set,
                                              batch_size=4,
                                              shuffle=True,
                                              num_workers=2)


validation_loader = torch.utils.data.DataLoader(validation_set,
                                                batch_size=4,
                                                shuffle=False,
                                                num_workers=2)

# Class labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Helper function for inline image display
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

# Extract a batch of 4 images
dataiter = iter(training_loader)
images, labels = next(dataiter)

# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

Above, we used TorchVision and Matplotlib to create a visual grid of a
minibatch of our input data. Below, we use the `add_image()` call on
`SummaryWriter` to log the image for consumption by TensorBoard, and we
also call `flush()` to make sure it's written to disk right away.


In [5]:
# Default log_dir argument is "runs" - but it's good to be specific
# torch.utils.tensorboard.SummaryWriter is imported above
writer = SummaryWriter('runs/fashion_mnist_experiment_1')

# Write image data to TensorBoard log dir
writer.add_image('Four Fashion-MNIST Images', img_grid)
writer.flush()

# To view, start TensorBoard on the command line with:
#   tensorboard --logdir=runs
# ...and open a browser tab to http://localhost:6006/

If you start TensorBoard at the command line and open it in a new
browser tab (usually at [localhost:6006](localhost:6006)), you should
see the image grid under the IMAGES tab.

Graphing Scalars to Visualize Training
======================================

TensorBoard is useful for tracking the progress and efficacy of your
training. Below, we'll run a training loop, track some metrics, and save
the data for TensorBoard's consumption.

Let's define a model to categorize our image tiles, and an optimizer and
loss function for training:


In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Now let's train a single epoch, and evaluate the training vs. validation
set losses every 1000 batches:


In [7]:
print(len(validation_loader))
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0

    for i, data in enumerate(training_loader, 0):
        # basic training loop
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:    # Every 1000 mini-batches...
            print('Batch {}'.format(i + 1))
            # Check against the validation set
            running_vloss = 0.0

            # In evaluation mode some model specific operations can be omitted eg. dropout layer
            net.train(False) # Switching to evaluation mode, eg. turning off regularisation
            for j, vdata in enumerate(validation_loader, 0):
                vinputs, vlabels = vdata
                voutputs = net(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss.item()
            net.train(True) # Switching back to training mode, eg. turning on regularisation

            avg_loss = running_loss / 1000
            avg_vloss = running_vloss / len(validation_loader)

            # Log the running loss averaged per batch
            writer.add_scalars('Training vs. Validation Loss',
                            { 'Training' : avg_loss, 'Validation' : avg_vloss },
                            epoch * len(training_loader) + i)

            running_loss = 0.0
print('Finished Training')

writer.flush()

2500
Batch 1000
Batch 2000
Batch 3000
Batch 4000
Batch 5000
Batch 6000
Batch 7000
Batch 8000
Batch 9000
Batch 10000
Batch 11000
Batch 12000
Batch 13000
Batch 14000
Batch 15000
Finished Training


Switch to your open TensorBoard and have a look at the SCALARS tab.

Visualizing Your Model
======================

TensorBoard can also be used to examine the data flow within your model.
To do this, call the `add_graph()` method with a model and sample input:


In [8]:
# Again, grab a single mini-batch of images
dataiter = iter(training_loader)
images, labels = next(dataiter)

# add_graph() will trace the sample input through your model,
# and render it as a graph.
writer.add_graph(net, images)
writer.flush()

When you switch over to TensorBoard, you should see a GRAPHS tab.
Double-click the "NET" node to see the layers and data flow within your
model.

Visualizing Your Dataset with Embeddings
========================================

The 28-by-28 image tiles we're using can be modeled as 784-dimensional
vectors (28 \* 28 = 784). It can be instructive to project this to a
lower-dimensional representation. The `add_embedding()` method will
project a set of data onto the three dimensions with highest variance,
and display them as an interactive 3D chart. The `add_embedding()`
method does this automatically by projecting to the three dimensions
with highest variance.

Below, we'll take a sample of our data, and generate such an embedding:


In [9]:
# Select a random subset of data and corresponding labels
def select_n_random(data, labels, n=100):
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# Extract a random subset of data
images, labels = select_n_random(training_set.data, training_set.targets)

# get the class labels for each image
class_labels = [classes[label] for label in labels]

# log embeddings
features = images.view(-1, 28 * 28)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images.unsqueeze(1))
writer.flush()
writer.close()

在 PyTorch 中，`torch.randperm(n)` 的功能是生成一个从  到  的**随机打乱的整数序列**（即输入 ，返回一个长度为  的随机排列张量）。

在你提供的代码片段中，它的核心作用是实现数据的**随机抽样（Random Sampling）**。

---

### 一、 `torch.randperm` 的基本用法

当你调用 `torch.randperm(len(data))` 时：

1. **输入**：`len(data)`，即数据集的总样本数（假设为 ）。
2. **输出**：一个形状为  的张量，包含  这些数字，但顺序是随机掉换的。

**简单示例：**

```python
import torch
perm = torch.randperm(5)
print(perm)
# 输出可能是: tensor([3, 0, 4, 1, 2])

```

---

### 二、 代码逻辑详细拆解

这段代码组合使用随机索引来实现高效的样本抽取：

#### 1. 生成随机索引序列

`perm = torch.randperm(len(data))`

* 这行代码创建了一个“随机索引表”。每一个数字代表原始 `data` 中某个样本的下标。

#### 2. 利用索引进行重排与切片

`return data[perm][:n], labels[perm][:n]`

* **`data[perm]`**：这是 PyTorch 的**高级索引（Advanced Indexing）**特性。它会根据 `perm` 中的随机顺序重新排列 `data` 中的所有样本。
* **`[:n]`**：从打乱后的数据中取出前 `n` 个。
* **同步性**：由于 `data` 和 `labels` 使用的是**同一个**随机索引序列 `perm`，这保证了特征和标签之间的对应关系（一一对应）不会被弄乱。

---

### 三、 为什么不直接用 `random.sample`？

在处理深度学习任务（尤其是涉及 PyTorch 张量）时，使用 `torch.randperm` 有以下优势：

1. **性能极高**：它直接在内存中操作张量，比 Python 原生的 `for` 循环或 `random` 库快得多。
2. **GPU 兼容**：如果你正在使用 GPU 训练（正如你之前的代码通过 `run_on_gpu` 判断的那样），`randperm` 可以直接在显存中生成，避免数据在 CPU 和 GPU 之间来回搬运。
3. **矩阵化操作**：非常适合配合批量处理（Batch Processing），是手写数据加载器（DataLoader）时的常用手段。

---

### 💡 进阶提示

这种方法适用于数据量能**一次性全部读入内存**的情况。如果你处理的是像你之前整理的“计网实验”资料中那样庞大的数据集，直接对整个数据集进行 `randperm` 可能会非常消耗内存。

**如果你在处理大型数据集时遇到了内存溢出的问题，需要我为你演示如何结合 `DataLoader` 和 `sampler` 来实现更节省内存的随机读取吗？**

在 PyTorch 中，`data[perm]` 利用了**张量映射**的原理。它并不是在原有的 `data` 上进行位置交换，而是根据你提供的“索引清单”去原始数据中提取对应内容，从而合成一个全新的张量。

以下是它具体的工作机制：

### 1. 映射机制：按“地址”抓取

你可以把 `data` 想象成一排带编号的抽屉（索引  到 ），而 `perm` 是一张写满了数字的**提货单**。

* 当你执行 `data[perm]` 时，系统会按照 `perm` 中数字出现的先后顺序，依次去 `data` 中找到对应编号的抽屉。
* **具体过程**：如果 `perm` 的第一个数是 ，系统就去拿 `data` 的第  个元素（索引为 ）；如果 `perm` 的第二个数是 ，系统就去拿 `data` 的第  个元素。

### 2. 空间逻辑：创建副本

* **非原地操作**：这种索引方式会产生一份**数据的拷贝（Copy）**。它会在内存中开辟一块新空间，将抓取到的数据按新顺序填进去。
* **多维处理**：如果 `data` 是一个二维矩阵（比如 `(样本数, 特征数)`），`data[perm]` 默认作用于**第 0 维**。它会把每一行作为一个整体进行重新排列，而每一行内部的特征顺序保持不变。

### 3. 图解示例

假设你的数据和随机索引如下：

* **`data`**: `['A', 'B', 'C', 'D']`（索引为 0, 1, 2, 3）
* **`perm`**: `tensor([2, 0, 3, 1])`

执行 `data[perm]` 的排列逻辑为：

1. 看 `perm[0]`：是 **2**  去找 `data[2]` ('C')  放在新张量的第 0 位。
2. 看 `perm[1]`：是 **0**  去找 `data[0]` ('A')  放在新张量的第 1 位。
3. 看 `perm[2]`：是 **3**  去找 `data[3]` ('D')  放在新张量的第 2 位。
4. 看 `perm[3]`：是 **1**  去找 `data[1]` ('B')  放在新张量的第 3 位。

* **最终结果**：`['C', 'A', 'D', 'B']`

### 4. 为什么这样能保持特征与标签同步？

在你的代码中，`data[perm]` 和 `labels[perm]` 使用的是**同一个 `perm` 变量**。
由于两个操作遵循完全相同的“提货单”顺序，即使数据被打乱了，每一行特征抓取的索引位置依然与它对应的标签索引位置严格对齐。

**你想了解这种高级索引在处理大规模数据（内存不足以一次性装下整个 `data`）时，有哪些更节省内存的替代方案吗？**

Now if you switch to TensorBoard and select the PROJECTOR tab, you
should see a 3D representation of the projection. You can rotate and
zoom the model. Examine it at large and small scales, and see whether
you can spot patterns in the projected data and the clustering of
labels.

For better visibility, it's recommended to:

-   Select "label" from the "Color by" drop-down on the left.
-   Toggle the Night Mode icon along the top to place the light-colored
    images on a dark background.

Other Resources
===============

For more information, have a look at:

-   PyTorch documentation on
    [torch.utils.tensorboard.SummaryWriter](https://pytorch.org/docs/stable/tensorboard.html?highlight=summarywriter)
-   Tensorboard tutorial content in the [PyTorch.org
    Tutorials](https://pytorch.org/tutorials/)
-   For more information about TensorBoard, see the [TensorBoard
    documentation](https://www.tensorflow.org/tensorboard)
